This uses the Jupyter Racket kernel from https://github.com/rmculpepper/iracket.

In [1]:
#lang iracket/lang #:require sicp

## Q1.1

In [2]:
10
;; 10

10

In [3]:
(+ 5 3 4)
;; 12

12

In [4]:
(- 9 1)
;; 8

8

In [5]:
(/ 6 2)
;; 3

3

In [6]:
(+ (* 2 4) (- 4 6))
;; 6

6

In [7]:
(define a 3)
(define b (+ a 1))
(+ a b (* a b))
;; 19

19

In [8]:
(= a b)
;; #f

#f

In [9]:
(if (and (> b a) (< b (* a b)))
    b
    a)
;; 4

4

In [10]:
(cond ((= a 4) 6)
      ((= b 4) (+ 6 7 a))
      (else 25))
;; 16

16

In [11]:
(+ 2 (if (> b a) b a))
;; 6

6

In [12]:
(* (cond ((> a b) a)
         ((< a b) b)
         (else -1))
   (+ a 1))
;; 16

16

## Q1.2

In [13]:
(/ (+ 5 4 (- 2 (- 3 (+ 6 (/ 4 5)))))
   (* 3 (- 6 2) (- 2 7)))

-37/150

## Q1.3

In [14]:
(define (sum s) (apply + s))
(define (square x) (* x x))
(define (sum-squared-largest a b c)
    (define minval (min a b c))
    (define vals (list a b c))
    (- (sum (map square vals)) (* minval minval)))

(sum-squared-largest 2 3 4)

25

## Q1.4

```python
# return a + |b|
def a_plus_abs_b(a, b):
    if b > 0:
        return a + b
    else:
        return a - b
```

## Q1.5

Applicative-order evaluation doesn't terminate. When we expand, we get
```scheme
(test 0 (p))
(test 0 (p))
...
```
since the interpreter keeps trying to reduce the recursive operand `(p)` first.


Normal-order evaluation returns 0 since it short-circuits the recursive evaluation by reducing the operator first.
```scheme
(test 0 (p))
(if (= 0 0) 0 (p))
(if #t 0 (p))
0
```

## Q1.6

Under the normal special form `if` statement, the predicate gets evaluated before the other arguments despite using applicative-order evaluation.
```scheme
;; Normal if
(sqrt-iter 1 2)
(if (good-enough? 1 2)
    1
    (sqrt-iter (improve 1 2) 2)))
(if #f
    1
    (sqrt-iter (improve 1 2) 2)))
(sqrt-iter (improve 1 2) 2)
(sqrt-iter 1.5 2)
...
```

New-if is instead treated as an ordinary procedure, so the `(sqrt-iter (improve guess x) x)` alternative gets infinitely expanded.
```scheme
;; New if
(sqrt-iter 1 2)
(new-if (good-enough? 1 2)
    1
    (sqrt-iter (improve 1 2) 2)))
(new-if (good-enough? 1 2)
    1
    (sqrt-iter 1.5 2)))
(new-if (good-enough? 1 2)
    1
    (new-if (good-enough? 1.5 2)
        1.5
        (sqrt-iter (improve 1.5 2) 2)))))
...
```

## Q1.7

`sqrt(0.0001)` is evaluated as 0.0323, significantly off from the its true value of 0.01. Here the tolerance of 0.001 is too high.\
`sqrt(1e13)` does not terminate because the improve step runs out of precision to improve while the difference remains above 0.001.

In [15]:
(define (average x y) (/ (+ x y) 2))
(define (square x) (* x x))
(define (improve guess x) (average guess (/ x guess)))
(define (good-enough? guess x) (< (abs (- (square guess) x)) 0.001))
(define (sqrt-iter guess x)
  (if (good-enough? guess x)
      guess
      (sqrt-iter (improve guess x) x)))
(define (sqrt x) (sqrt-iter 1.0 x))
(display (sqrt 0.0001)) (newline)

0.03230844833048122


In [16]:
(define (average x y) (/ (+ x y) 2))
(define (square x) (* x x))
(define (improve guess x) (average guess (/ x guess)))
(define (good-enough? guess x) (and
        (display guess)
        (display " ")
        (display (abs (- (square guess) x))) (newline)
        (= guess (improve guess x))))
(define (sqrt-iter guess x)
  (if (good-enough? guess x)
      guess
      (sqrt-iter (improve guess x) x)))
(define (sqrt x) (sqrt-iter 1.0 x))
(display (sqrt 1e13)) (newline)

1.0 9999999999999.0
5000000000000.5 2.4999999999995e+25
2500000000001.25 6.24999999999625e+24
1250000000002.625 1.5624999999965625e+24
625000000005.3125 3.906249999966406e+23
312500000010.65625 9.765624999666016e+22
156250000021.32813 2.441406249666504e+22
78125000042.66406 6.103515621666259e+21
39062500085.33203 1.525878902916565e+21
19531250170.666016 3.814697232291413e+20
9765625341.333006 9.536742830728538e+19
4882813182.666485 2.384185457682161e+19
2441407615.3330994 5960461144206451000.0
1220705855.6654043 1490112786055807000.0
610357023.8235396 372525696530728900.0
305186703.8384698 93128924199789870.0
152609735.33285388 23279731318363710.0
76337630.97598314 5817433903025381.0
38234313.983767174 1451862765809294.0
19247929.57491261 360482792920795.56
9883732.98644831 87688177747406.44
5447748.229088079 19677960767532.305
3641684.517987917 3261866128552.8887
3193833.2403111905 200570766916.67773
3162433.547242504 985940724.8066406
3162277.6640104805 24299.58203125
3162277.6601683

Stopping when the guess's change becomes relatively small helps in both cases.

In [17]:
(define (average x y) (/ (+ x y) 2))
(define (square x) (* x x))
(define (improve guess x) (average guess (/ x guess)))
(define (good-enough? guess x) (
        < (abs (- guess (improve guess x))) (* 0.001 guess)))
(define (sqrt-iter guess x)
  (if (good-enough? guess x)
      guess
      (sqrt-iter (improve guess x) x)))
(define (sqrt x) (sqrt-iter 1.0 x))
(display (sqrt 0.0001)) (newline)
(display (sqrt 1e13)) (newline)

0.010000714038711746
3162433.547242504


## Q1.8

In [18]:
(define (average x y) (/ (+ x y) 2))
(define (square x) (* x x))
(define (improve guess x) (/ (+ (/ x (square guess)) (* 2 guess)) 3))
(define (good-enough? guess x) (
        < (abs (- guess (improve guess x))) (* 0.001 guess)))
(define (cubrt-iter guess x)
  (if (good-enough? guess x)
      guess
      (cubrt-iter (improve guess x) x)))
(define (cubrt x) (cubrt-iter 1.0 x))
(display (cubrt 0.001)) (newline)
(display (cubrt 1e12)) (newline)
(display (cubrt 1e13)) (newline)

0.10001409266436927
10001.829838573718
21545.34186023313


## Q1.9

The first procedure generates a recursive process.
```scheme
(define (+ a b)
    (if (= a 0) b (inc (+ (dec a) b))))
(+ 4 5)
(if (= 4 0) 5 (inc (+ (dec 4) 5)))
(if #f 5 (inc (+ (dec 4) 5)))
(inc (+ (dec 4) 5))
(inc (+ 3 5))
(inc (if (= 3 0) 5 (inc (+ (dec 3) 5))))
(inc (inc (+ (dec 3) 5)))
(inc (inc (+ 2 5)))
...
(inc (inc (inc (inc (+ 0 5)))))
(inc (inc (inc (inc 5))))
...
9
```

The second procedure generates an iterative process.
```scheme
(define (+ a b)
    (if (= a 0) b (+ (dec a) (inc b))))
(+ 4 5)
(if (= 4 0) 5 (+ (dec 4) (inc 5)))
(if #f 5 (+ (dec 4) (inc 5)))
(+ (dec 4) (inc 5))
(+ 3 6)
(if (= 3 0) 6 (+ (dec 3) (inc 6)))
(+ (dec 3) (inc 6))
(+ 2 7)
...
(+ 0 9)
9
```

## Q1.10

In [19]:
(define (A x y) (cond ((= y 0) 0)
    ((= x 0) (* 2 y))
    ((= y 1) 2)
    (else (A (- x 1) (A x (- y 1))))))

#|
A(0, 0) = 0
A(1, 0) = 0
A(2, 0) = 0
A(3, 0) = 0
A(0, 1) = 2
A(0, 2) = 4
A(0, 3) = 6
A(1, 1) = 2
A(1, 2) = A(0, A(1, 1)) = A(0, 2) = 4
A(1, 3) = A(0, A(1, 2)) = A(0, 4) = 8
A(1, 10) = A(0, A(1, 9)) = A(0, 2^9) = 2^10 = 1024
A(2, 1) = 2
A(2, 2) = A(1, A(2, 1)) = A(1, 2) = 4
A(2, 3) = A(1, A(2, 2)) = A(1, 4) = 16
A(2, 4) = A(1, A(2, 3)) = A(1, 16) = 2^16 = 65536
A(3, 1) = 2
A(3, 2) = A(2, A(3, 1)) = A(2, 2) = 4
A(3, 3) = A(2, A(3, 2)) = A(2, 4) = 65536
|#

(display (A 1 10)) (newline) ; 1024
(display (A 2 4)) (newline) ; 65536
(display (A 3 3)) (newline) ; 65536

1024
65536
65536


In [20]:
(define (f n) (A 0 n))
;; f(n) = 2n
(display (f 100)) (newline) ; 200

(define (g n) (A 1 n))
;; g(n) = 2^n
(display (g 5)) (newline) ; 32

(define (h n) (A 2 n))
;; h(n) = 2^^n
(display (h 4)) (newline) ; 65536

(define (k n) (* 5 n n))

200
32
65536


## Q1.11

In [21]:
;; recursive since multiple terms expand in the else-case
(define (f n)
  (if (< n 3)
      n
      (+ (f (- n 1)) (* 2 (f (- n 2))) (* 3 (f (- n 3))))))

;; f(5) = f(4)+2f(3)+3f(2) = 3f(3)+5f(2)+3f(1) = 8f(2)+9f(1)+9f(0) = 16+9 = 25
(f 5)

25

In [22]:
;; iterative
(define (f n)
    ;; (fi n a b c) computes a*f(n-1) + b*f(n-2) + c*f(n-3)
    (define (fi n a b c)
        (if (= n 3)
            (+ (* a 2) (* b 1) (* c 0)) 
            (fi (- n 1) (+ b a) (+ c (* 2 a)) (* 3 a)))) 
    (if (< n 3)
        n
        (fi n 1 2 3)))
(f 5)

25

## Q1.12

In [23]:
;; 0-indexed
(define (pascal n k)
  (cond ((or (< n 0) (< k 0) (< n k)) 0)
        ((= k 0) 1)
        ((= k n) 1)
        (else (+ (pascal (- n 1) (- k 1))
                 (pascal (- n 1) k)))))

(display (pascal 0 0)) (newline)
(display (pascal 3 5)) (newline)
(display (pascal 4 2)) (newline)
(display (pascal 5 3)) (newline)

1
0
6
10


## Q1.13

Let $\phi = (1+\sqrt{5})/2$ and $\psi = (1-\sqrt{5})/2$.

Base cases: $\text{Fib}(0) = (\phi^0-\psi^0)/\sqrt{5} = 0$, $\text{Fib}(1) = (\phi^1-\psi^1)/\sqrt{5} = 1$.

Suppose $\text{Fib}(i) = (\phi^i-\psi^i)/\sqrt{5}$ for all $0 \le i \le k$.

Then starting with the desired formula for $\text{Fib}(k+1)$,
$$
\begin{align}
(\phi^{k+1}-\psi^{k+1})/\sqrt{5}
&= \frac{1}{\sqrt{5}} \left(\phi^{k} + \phi^{k}(\phi-1) - \psi^{k} - \psi^{k}(\psi-1)\right) \\
&= \frac{1}{\sqrt{5}} \left(\phi^{k} + \phi^{k-1}\phi(\phi-1) - \psi^{k} - \psi^{k-1}\psi(\psi-1)\right) \\
\end{align}
$$.

Note that
$\phi(\phi-1) = \left(\frac{1+\sqrt{5}}{2}\right) \left(\frac{-1+\sqrt{5}}{2}\right) = \frac{5-1}{4} = 1$
and
$\psi(\psi-1) = \left(\frac{1-\sqrt{5}}{2}\right) \left(\frac{-1-\sqrt{5}}{2}\right) = \frac{5-1}{4} = 1$.

Thus the above expression simplies to
$(\phi^{k+1}-\psi^{k+1})/\sqrt{5} = \frac{1}{\sqrt{5}} \left(\phi^{k} + \phi^{k-1} - \psi^{k} - \psi^{k-1}\right) = \text{Fib}(k) + \text{Fib}(k-1) = \text{Fib}(k+1)$
as desired.

Now we need to show that $\text{Fib}(n) = (\phi^n-\psi^n)/\sqrt{5}$ is the closest integer to $\phi^n/\sqrt{5}$.

Since $\text{Fib}(n)$ is an integer, this would only be untrue if $|\psi^n/\sqrt{5}| > \frac{1}{2}$ or $|\psi^n| > \frac{\sqrt{5}}{2}$.

We note that $|\psi| < 1$ and thus $|\psi^n| < 1 < \frac{\sqrt{5}}{2}$.

## Q1.14

```scheme
(count-change 11)
(cc 11 5)
(+ (cc 11 4) (cc -39 5))
(+ (+ (cc 11 3) (cc -14 4)) (cc -39 5))
(+ (+ (+ (cc 11 2) (cc 1 3)) (cc -14 4)) (cc -39 5))
(+ (+ (+ (+ (cc 11 1) (cc 6 2)) (cc 1 3)) (cc -14 4)) (cc -39 5))
(+ (+ (+ (+ (+ (cc 11 0) (cc 10 1)) (cc 6 2)) (cc 1 3)) (cc -14 4)) (cc -39 5))
(+ (+ (+ (+ (+ 0 (cc 10 1)) (cc 6 2)) (cc 1 3)) (cc -14 4)) (cc -39 5))
(+ (+ (+ (+ (+ 0 (+ (cc 10 0) (cc 9 1))) (cc 6 2)) (cc 1 3)) (cc -14 4)) (cc -39 5))
...
(+ (+ (+ (+ (+ 0 (+ ... (+ (cc 1 0) (cc 0 1)) ... )) (cc 6 2)) (cc 1 3)) (cc -14 4)) (cc -39 5))
(+ (+ (+ (+ (+ 0 (+ ... (+ 0 1) ... )) (cc 6 2)) (cc 1 3)) (cc -14 4)) (cc -39 5))
(+ (+ (+ (+ (+ 0 (+ ... (+ 0 1) ... )) (+ (cc 6 1) (+ cc 1 2))) (cc 1 3)) (cc -14 4)) (cc -39 5))
...
4
```

We can verify the call tree by printing it.

In [24]:
(define (print-tup a b)
    (display "(")
    (display a)
    (display ", ")
    (display b)
    (display ")"))

In [25]:
(define (count-change amount) (cc amount 5))
(define (cc amount kinds-of-coins)
    (print-tup (number->string amount) (number->string kinds-of-coins))
    (cond ((= amount 0) 1)
          ((or (< amount 0) (= kinds-of-coins 0)) 0) 
           (else (+ (cc amount (- kinds-of-coins 1))
                    (cc (- amount (first-denomination kinds-of-coins)) kinds-of-coins)))))
(define (first-denomination kinds-of-coins)
    (cond ((= kinds-of-coins 1) 1) 
          ((= kinds-of-coins 2) 5)
          ((= kinds-of-coins 3) 10)
          ((= kinds-of-coins 4) 25)
          ((= kinds-of-coins 5) 50)))

(count-change 11)

(11, 5)(11, 4)(11, 3)(11, 2)(11, 1)(11, 0)(10, 1)(10, 0)(9, 1)(9, 0)(8, 1)(8, 0)(7, 1)(7, 0)(6, 1)(6, 0)(5, 1)(5, 0)(4, 1)(4, 0)(3, 1)(3, 0)(2, 1)(2, 0)(1, 1)(1, 0)(0, 1)(6, 2)(6, 1)(6, 0)(5, 1)(5, 0)(4, 1)(4, 0)(3, 1)(3, 0)(2, 1)(2, 0)(1, 1)(1, 0)(0, 1)(1, 2)(1, 1)(1, 0)(0, 1)(-4, 2)(1, 3)(1, 2)(1, 1)(1, 0)(0, 1)(-4, 2)(-9, 3)(-14, 4)(-39, 5)

4

Let $T(n, k)$ denote the number of operations needed to make change for $n$ cents using our smallest $k$ denominations.

Space: $\Theta(n)$ since we are effectively doing a post-order traversal of the call tree which has depth $n$ (from using the smallest denomination).

Time: $\Theta(n^k)$.

For time, $T(n, 1) = 2n + 1$ since we make calls `cc(n, 1), cc(n-1, 1), ..., cc(0, 1)` and `cc(n, 0), cc(n-1, 0), ..., cc(1, 0)`.

$T(n, 2)$ makes "use nickels" calls $k=\lceil n/5 \rceil + 1$ times `cc(n, 2), cc(n-5, 2), ..., cc(n-5(k-1), 2)`. Each of these $k-1$ calls `cc(i, 2)` (we exclude `cc(n-5(k-1), 2)` which ran out of money) subsequently generates $T(i, 1)$ calls that try to use only pennies.

Thus
$$
\begin{align}
T(n, 2) &= \sum_{j=0}^{k-2} T(n-5j, 1) + 1 \\
&= \sum_{j=0}^{k-2} (2(n-5j)+1) + 1 \\
&= 1 + (k-1) + \sum_{j=0}^{k-2} 2n - \sum_{j=0}^{k-2} 10j \\
&= 1 + \lceil n/5 \rceil + 2n(\lceil n/5 \rceil) - 10(\lceil n/5 \rceil - 1)(\lceil n/5 \rceil)/2 \\
&= \Theta(n^2)
\end{align}
$$.

Similarly, $T(n, 3)$ makes "use dimes" calls $\Theta(n/10)$ times that each make $T(i, 2) = \Theta(n^2)$ further calls.

So $T(n, 3) = \Theta(n^3)$ and $T(n, k) = \Theta(n^k)$ in general.

## Q1.15

In [26]:
(define (cube x) (* x x x))
(define (p x)
    (- (* 3 x) (* 4 (cube x))))
(define (sine angle)
    (if (not (> (abs angle) 0.1))
        angle
        (and
            (display angle) (newline)
            (p (sine (/ angle 3.0))))))

(sine 12.15)

12.15
4.05
1.3499999999999999
0.44999999999999996
0.15


-0.39980345741334

`p` is applied 5 times.

Each iteration divides the angle $a$ by 3 until $a \le 0.1$, that is, until $\frac{a}{3^n} \le 0.1$.

This means we need $\log_{3}{\frac{a}{0.1}}$ steps and both the time and space grow with $\Theta(\log(a))$.

In [27]:
(define (steps a)
  (/ (log (/ a 0.1)) (log 3)))
(steps 12.15)

4.369070246428542

## Q1.16

In [28]:
(define (fast-expt b n)
    (define (fe b n acc)
        (cond ((= n 0) acc)
              ((even? n) (fe (* b b) (/ n 2) acc))
              (else (fe b (- n 1) (* acc b)))))
    (fe b n 1))

(display (fast-expt 2 10)) (newline)
(display (fast-expt 2 16)) (newline)
(display (fast-expt 2 31)) (newline)

1024
65536
2147483648


## Q1.17

In [29]:
(define (double x) (* x 2))
(define (half x) (/ x 2))
(define (mul a b)
    (cond ((= b 0) 0)
          ((even? b) (mul (double a) (half b)))
          (else (+ a (mul a (- b 1))))))

(display (mul 3 7)) (newline)
(display (mul 3 20)) (newline)

21
60


## Q1.18

In [30]:
(define (double x) (* x 2))
(define (half x) (/ x 2))
(define (mul a b)
    (define (mul-it a b acc)
        (cond ((= b 0) acc)
              ((even? b) (mul-it (double a) (half b) acc))
              (else (mul-it a (- b 1) (+ acc a)))))
    (mul-it a b 0))

(display (mul 3 7)) (newline)
(display (mul 3 20)) (newline)

21
60


## Q1.19

We are basically saying that
$
T_{pq} =
\begin{bmatrix}
q+p & q \\
q     & p
\end{bmatrix}
$
such that
$
T_{pq}
\begin{bmatrix}
a \\
b
\end{bmatrix}
=
\begin{bmatrix}
qa+pq+qb \\
qa+pb
\end{bmatrix}
$.

Thus
$
T_{pq}^2 =
\begin{bmatrix}
(q+p)^2+q^2 & q(q+p)+qp \\
q(q+p)+qp   & q^2+p^2
\end{bmatrix}
=
\begin{bmatrix}
(q^2+2pq)+(q^2+p^2) & (q^2+2pq) \\
(q^2+2pq)           & (q^2+p^2)
\end{bmatrix}
=
\begin{bmatrix}
q'+p' & q' \\
q'    & p'
\end{bmatrix}
$.

In [31]:
(define (fib n)
    (fib-iter 1 0 0 1 n))
(define (fib-iter a b p q count)
    (cond ((= count 0) b)
          ((even? count)
           (fib-iter a
                     b
                     (+ (* q q) (* p p))
                     (+ (* q q) (* 2 p q))
                     (/ count 2)))
          (else (fib-iter (+ (* b q) (* a q) (* a p))
                          (+ (* b p) (* a q))
                          p
                          q
                          (- count 1)))))
          
(display (fib 4)) (newline)
(display (fib 5)) (newline)
(display (fib 6)) (newline)
(display (fib 20)) (newline)

3
5
8
6765


In [32]:
(define sqrt5 (expt 5 0.5))
(define phi (/ (+ 1 sqrt5) 2))
(define psi (/ (- 1 sqrt5) 2))
(define (fib-check n)
    (/ (- (expt phi n) (expt psi n)) sqrt5))

(display (fib-check 4)) (newline)
(display (fib-check 5)) (newline)
(display (fib-check 6)) (newline)
(display (fib-check 20)) (newline)

3.0000000000000004
5.000000000000001
8.000000000000002
6765.000000000005


## Q1.20

In [33]:
(define (gcd a b)
    (if (= b 0)
        a
        (gcd b (remainder a b))))

(gcd 9 6)

3

Applicative-order:
```scheme
(gcd 206 40)
(gcd 40 (remainder 206 40))
(gcd 40 6)
(gcd 6 (remainder 40 6))
(gcd 6 4)
(gcd 4 (remainder 6 4))
(gcd 4 2)
(gcd 2 (remainder 4 2))
(gcd 2 0)
2
```
4 calls to `remainder`.

Normal-order:
```scheme
(gcd 206 40)
(if (= 40 0)  ; #f
    206  ; not evaluated
    (gcd 40 (remainder 206 40)))
(gcd 40 (remainder 206 40))  ; #f
(if (= (gcd 40 (remainder 206 40)) 0)
    40  ; not evaluated
    (gcd
        (gcd 40 (remainder 206 40))
        (remainder
            40
            (gcd 40 (remainder 206 40))))
...
(if (= b' 0)  ; #t
    a'   ; evaluated
    b')
```

In normal-order evaluation, `remainder` primarily gets evaluated when substituting `b` in `gcd`'s `(if (= b 0))` check. This continues until the remainder is 0 whereupon we evaluate any `remainder` instances in the `a` branch of the if-statement.

In each step of the Euclidean algorithm, we set `b' = (remainder a b)`, `a' = b` and call `(gcd a' b')`.

Denote the number of `remainder` calls in the $i$th iteration `b` as $c(b_i)$. We start with $c(b_0) = c(a_0) = 0$.

We can see from our update rule that $c(b_{i+1}) = 1 + c(a_i) + c(b_i)$ and $c(a_{i+1}) = c(b_i)$.

After $i$ steps, we will have made $c(a_i) + \sum_{j=0}^i c(b_j)$ = $c(b_{i-1}) + \sum_{j=0}^i c(b_j)$ calls in total.

The first terms of $c(a_i)$ are 0, 0, 1, 2, 4.

The first terms of $c(b_i)$ are 0, 1, 2, 4, 7.

So we make 1+2+4+7+4 = 18 calls to `remainder`.

**Aside:** since $c(b_{i})$ is a linear recurrence relation, we can solve it using standard methods. We homogenize it as $c(b_{i}) = 2c(b_{i-1}) - c(b_{i-3})$ and find that $c(b_{t}) = k_1 \lambda_1^t + k_2 \lambda_2^t + k_3 \lambda_3^t$ with $\lambda_1 = 1$, $\lambda_2 = \phi = (1+\sqrt{5})/2$, and $\lambda_3 = \psi = (1-\sqrt{5})/2$.

Using our initial conditions, we find that $k_1 = 1$, $k_2 = \frac{\sqrt{5}+1}{2\sqrt{5}}$, and $k_3 = \frac{\sqrt{5}-1}{2\sqrt{5}}$.

Now we can compute our partial sums $\sum_{j=0}^i c(b_j) = k_1 \sum_{j=1}^{i+1} \lambda_1^j + k_2 \sum_{j=1}^{i+1} \lambda_2^j + k_3 \sum_{j=1}^{i+1} \lambda_3^j$ without recurrence by using the formula $\sum_{i=0}^{n-1} r^i = \frac{1-r^n}{1-r}$ for $r \ne 1$.

Maybe there's an easier way...I tried putting the recurrence in matrix form and using the formula for a partial Neumann series but that involved inverting a singular matrix.

In [34]:
(define sqrt5 (expt 5 0.5))
(define phi (/ (+ 1 sqrt5) 2))
(define psi (/ (- 1 sqrt5) 2))
(define k1 -1)
(define k2 (/ (+ sqrt5 1) (* 2 sqrt5)))
(define k3 (/ (- sqrt5 1) (* 2 sqrt5)))
(define (geom-sum r n)
    ;; geometric sum of r^i for i={0..(n-1)}, need r != 1
    (/ (- 1 (expt r n))
       (- 1 r)))

(define (bterm n)
    ;; element c(b_n)
    (+ (* k1 1)
       (* k2 (expt phi (+ n 1)))
       (* k3 (expt psi (+ n 1)))))

(define (bsum n)
    ;; sum of c(b_i) for i={0..n}
    (+ (* k1 (+ n 2))
       (* k2 (geom-sum phi (+ n 2)))
       (* k3 (geom-sum psi (+ n 2)))))

(display "first c(b_i) terms:") (newline)
(display (bterm 0)) (newline)
(display (bterm 1)) (newline)
(display (bterm 2)) (newline)
(display (bterm 3)) (newline)
(display (bterm 4)) (newline)
(newline)
(display "first c(b_i) sums:") (newline)
(display (bsum 0)) (newline)
(display (bsum 1)) (newline)
(display (bsum 2)) (newline)
(display (bsum 3)) (newline)
(display (bsum 4)) (newline)
(newline)
(display "number of remainder calls:") (newline)
(display (+ (bsum 4) (bterm 3)))

first c(b_i) terms:
-1.3877787807814457e-16
0.9999999999999999
2.0
4.0
7.000000000000002

first c(b_i) sums:
-1.3877787807814457e-16
0.9999999999999998
3.0
7.000000000000002
14.000000000000002

number of remainder calls:
18.0

## Q1.21

In [35]:
(define (square x) (* x x))
(define (smallest-divisor n) (find-divisor n 2))
(define (find-divisor n test-divisor)
    (cond ((> (square test-divisor) n) n)
          ((divides? test-divisor n) test-divisor)
          (else (find-divisor n (+ test-divisor 1)))))
(define (divides? a b) (= (remainder b a) 0))

(display (smallest-divisor 199)) (newline)
(display (smallest-divisor 1999)) (newline)
(display (smallest-divisor 19999)) (newline)

199
1999
7


## Q1.22

In [36]:
(define (prime? n)
    (= n (smallest-divisor n)))
(define (timed-prime-test? n)
    (newline)
    (display n)
    (start-prime-test? n (runtime)))
(define (start-prime-test? n start-time)
    (if (prime? n)
        (report-prime? (- (runtime) start-time))
        #f))
(define (report-prime? elapsed-time)
    (display " *** ")
    (display elapsed-time)
    #t)

(define (search-for-primes start end target)
    (define (sfp-it curr end target)
        (cond ((or (> curr end) (= target 0)) (newline) (display "Done") (newline))
              ((= (remainder curr 2) 0) (sfp-it (+ 1 curr) end target))
              (else (if (timed-prime-test? curr)
                        (sfp-it (+ 2 curr) end (- target 1))
                        (sfp-it (+ 2 curr) end target)))))
    (sfp-it start end target))

(search-for-primes 1000 1100 3)
(search-for-primes 10000 11000 3)
(search-for-primes 100000 101000 3)
(search-for-primes 1000000 1001000 3)
(search-for-primes 100000000 100001000 3)
(search-for-primes 1000000000 1000001000 3)


1001
1003
1005
1007
1009 *** 1
1011
1013 *** 1
1015
1017
1019 *** 1
Done

10001
10003
10005
10007 *** 2
10009 *** 1
10011
10013
10015
10017
10019
10021
10023
10025
10027
10029
10031
10033
10035
10037 *** 2
Done

100001
100003 *** 3
100005
100007
100009
100011
100013
100015
100017
100019 *** 2
100021
100023
100025
100027
100029
100031
100033
100035
100037
100039
100041
100043 *** 2
Done

1000001
1000003 *** 8
1000005
1000007
1000009
1000011
1000013
1000015
1000017
1000019
1000021
1000023
1000025
1000027
1000029
1000031
1000033 *** 8
1000035
1000037 *** 7
Done

100000001
100000003
100000005
100000007 *** 68
100000009
100000011
100000013
100000015
100000017
100000019
100000021
100000023
100000025
100000027
100000029
100000031
100000033
100000035
100000037 *** 711
100000039 *** 69
Done

1000000001
1000000003
1000000005
1000000007 *** 832
1000000009 *** 918
1000000011
1000000013
1000000015
1000000017
1000000019
1000000021 *** 923
Done


Runtime is noisy but mostly as expected.

## Q1.23

In [37]:
(define (next test-divisor)
    (if (= test-divisor 2)
        3
        (+ test-divisor 2)))

(define (square x) (* x x))
(define (smallest-divisor n) (find-divisor n 2))
(define (find-divisor n test-divisor)
    (cond ((> (square test-divisor) n) n)
          ((divides? test-divisor n) test-divisor)
          (else (find-divisor n (next test-divisor)))))
(define (divides? a b) (= (remainder b a) 0))
(define (prime? n)
    (= n (smallest-divisor n)))
(define (timed-prime-test n)
    (newline)
    (display n)
    (start-prime-test n (runtime)))
(define (start-prime-test n start-time)
    (if (prime? n)
        (report-prime (- (runtime) start-time))))
(define (report-prime elapsed-time)
    (display " *** ")
    (display elapsed-time))

(timed-prime-test 1009)
(timed-prime-test 1013)
(timed-prime-test 1019)
(timed-prime-test 10007)
(timed-prime-test 10009)
(timed-prime-test 10037)
(timed-prime-test 100003)
(timed-prime-test 100019)
(timed-prime-test 100043)
(timed-prime-test 1000003)
(timed-prime-test 1000033)
(timed-prime-test 1000037)
(timed-prime-test 100000007)
(timed-prime-test 100000037)
(timed-prime-test 100000039)
(timed-prime-test 1000000007)
(timed-prime-test 1000000009)
(timed-prime-test 1000000021)


1009 *** 2
1013 *** 1
1019 *** 0
10007 *** 2
10009 *** 1
10037 *** 1
100003 *** 2
100019 *** 2
100043 *** 3
1000003 *** 6
1000033 *** 7
1000037 *** 5
100000007 *** 49
100000037 *** 49
100000039 *** 50
1000000007 *** 159
1000000009 *** 173
1000000021 *** 167

The results are still noisy but are mostly slower for small numbers. We messed with the component functions so it's hard to tell exactly why; in any case the `(if (= test-divisor 2))` check is not free.

## Q1.24

In [38]:
(define (square x) (* x x))
(define (expmod base exp m)
    (cond ((= exp 0) 1)
          ((even? exp)
           (remainder
                (square (expmod base (/ exp 2) m))
                m))
          (else
             (remainder
                (* base (expmod base (- exp 1) m))
                m))))
(define (fermat-test n)
    (define (try-it a)
        (= (expmod a n n) a))
    (try-it (+ 1 (random (- n 1)))))
(define (fast-prime? n times)
    (cond ((= times 0) true)
          ((fermat-test n) (fast-prime? n (- times 1)))
          (else false)))
(define (prime? n)
    (fast-prime? n 1000))
(define (timed-prime-test n)
    (newline)
    (display n)
    (start-prime-test n (runtime)))
(define (start-prime-test n start-time)
    (if (prime? n)
        (report-prime (- (runtime) start-time))))
(define (report-prime elapsed-time)
    (display " *** ")
    (display elapsed-time))

(timed-prime-test 1009)
(timed-prime-test 1013)
(timed-prime-test 1019)
(timed-prime-test 10007)
(timed-prime-test 10009)
(timed-prime-test 10037)
(timed-prime-test 100003)
(timed-prime-test 100019)
(timed-prime-test 100043)
(timed-prime-test 1000003)
(timed-prime-test 1000033)
(timed-prime-test 1000037)
(timed-prime-test 100000007)
(timed-prime-test 100000037)
(timed-prime-test 100000039)
(timed-prime-test 1000000007)
(timed-prime-test 1000000009)
(timed-prime-test 1000000021)


1009 *** 248
1013 *** 536
1019 *** 274
10007 *** 264
10009 *** 456
10037 *** 325
100003 *** 340
100019 *** 559
100043 *** 467
1000003 *** 492
1000033 *** 352
1000037 *** 377
100000007 *** 653
100000037 *** 578
100000039 *** 618
1000000007 *** 616
1000000009 *** 622
1000000021 *** 645

We expect logarithmic growth but don't really see it (I'm okay with this).

## Q1.25

This still works but is not as performant since it fully computes $a^n$ before taking the remainder. The original version takes remainders after multiplying by each factor so it deals with much smaller numbers.

## Q1.26

`expmod` is $\Theta(\log(n))$ because it uses $\log(n)$ successive squarings to compute $a^n$. By doubling the amount of work at each squaring, we end up doing $2^{\log(n)} = \Theta(n)$ operations.

Another way to see this is that expanding the call graph in Louis's code results in an expression that's equivalent to multiplying $a$ by itself $n$ times, i.e. a naive $\Theta(n)$ computation of $a^n$.

## Q1.27

In [39]:
(define (square x) (* x x))
(define (expmod base exp m)
    (cond ((= exp 0) 1)
          ((even? exp)
           (remainder
                (square (expmod base (/ exp 2) m))
                m))
          (else
             (remainder
                (* base (expmod base (- exp 1) m))
                m))))
(define (fermat-test n candidate)
    (define (try-it a)
        (= (expmod a n n) a))
    (try-it candidate))
(define (fast-prime? n candidate)
    (cond ((= candidate n) true)
          ((fermat-test n candidate) (fast-prime? n (+ candidate 1)))
          (else false)))
(define (prime? n)
    (fast-prime? n 2))
(define (timed-prime-test n)
    (newline)
    (display n)
    (start-prime-test n (runtime)))
(define (start-prime-test n start-time)
    (if (prime? n)
        (report-prime (- (runtime) start-time))))
(define (report-prime elapsed-time)
    (display " *** ")
    (display elapsed-time))

(timed-prime-test 555)
(timed-prime-test 560)
;; Carmichael below
(timed-prime-test 561)
(timed-prime-test 1105)
(timed-prime-test 1729)
(timed-prime-test 2465)
(timed-prime-test 2821)
(timed-prime-test 6601)


555
560
561 *** 67
1105 *** 142
1729 *** 219
2465 *** 533
2821 *** 468
6601 *** 1564

## Q1.28

This solution has rather ugly code and I might have interpreted the "nontrivial square root" condition wrong. Based on this [stackoverflow thread](https://stackoverflow.com/questions/55969284/sicp-exercise-1-28-miller-rabin-at-least-half-the-numbers-will-reveal-a-non), the book's original description is also wrong.

I don't care enough about Miller-Rabin to dig into it further right now, though.

In [40]:
(define (square-chk x n)
    ;; returns 0 if x is a nontrivial square root of 1 (mod n)
    (define square-val (* x x))
    (if (and (not (or (= x 1) (= x (- n 1))))
             (= 1 (remainder square-val n)))
        0
        square-val))
(define (expmod-chk base exp m)
    ;; returns 0 if we find a nontrivial square root of 1 (mod m)
    (cond ((= exp 0) 1)
          ((even? exp)
           (remainder
                (square-chk (expmod base (/ exp 2) m) m)
                m))
          (else
             (remainder
                (* base (expmod base (- exp 1) m))
                m))))
(define (fermat-test n candidate)
    (define (try-it a)
        (= (expmod-chk a (- n 1) n) 1))
    (try-it candidate))
(define (miller-rabin? n candidate)
    (cond ((> candidate (+ (/ n 2) 1)) true)
          ((fermat-test n candidate) (miller-rabin? n (+ candidate 1)))
          (else false)))
(define (prime? n)
    (miller-rabin? n 2))
(define (timed-prime-test n)
    (newline)
    (display n)
    (start-prime-test n (runtime)))
(define (start-prime-test n start-time)
    (if (prime? n)
        (report-prime (- (runtime) start-time))))
(define (report-prime elapsed-time)
    (display " *** ")
    (display elapsed-time))

(timed-prime-test 2)
(timed-prime-test 3)
(timed-prime-test 4)
(timed-prime-test 10)
(timed-prime-test 13)
(timed-prime-test 17)
(timed-prime-test 21)
(timed-prime-test 27)
(timed-prime-test 37)
(timed-prime-test 1009)
(timed-prime-test 1013)
(timed-prime-test 1019)
(timed-prime-test 10007)
(timed-prime-test 10009)
(timed-prime-test 10037)
;; Carmichael below
(timed-prime-test 560)
(timed-prime-test 561)
(timed-prime-test 1105)
(timed-prime-test 1729)
(timed-prime-test 2465)
(timed-prime-test 2821)
(timed-prime-test 6601)


2
3 *** 3
4
10
13 *** 2
17 *** 1
21
27
37 *** 3
1009 *** 90
1013 *** 94
1019 *** 553
10007 *** 1271
10009 *** 1236
10037 *** 1334
560
561
1105
1729
2465
2821
6601

## Q1.29

In [41]:
(define (sum term a next b)
    (if (> a b)
        0
        (+ (term a)
           (sum term (next a) next b))))
(define (cube x) (* x x x))
(define (integral f a b dx)
    (define (add-dx x)
        (+ x dx))
    (* (sum f (+ a (/ dx 2.0)) add-dx b)
       dx))

(define (simpson f a b n)
    (define h
        (/ (- b a) n))
    (define (y-coef k)
        (cond ((= k 0) 1)
              ((= k n) 1)
              ((even? k) 2)
              (else 4)))
    (define (term k)
        (* (y-coef k) (f (+ a (* k h)))))
    (define (next k)
        (+ k 1))
    (* (sum term a next n)
       (/ h 3)))

;; both are 100 terms
(display (integral cube 0 1 0.01)) (newline)
(display (simpson cube 0 1 100)) (newline)

;; both are 1000 terms
(display (integral cube 0 1 0.001)) (newline)
(display (simpson cube 0 1 1000)) (newline)

0.24998750000000042
1/4
0.249999875000001
1/4


Simpson's rule is more accurate.

## Q1.30

In [42]:
(define (sum term a next b)
    (define (iter a result)
        (if (> a b)
            result
            (iter (next a) (+ result (term a)))))
    (iter a 0))

(define (identity k) k)
(define (inc k) (+ k 1))
(sum identity 0 inc 5)  ; 15

15

## Q1.31

In [43]:
;; a
(define (product term a next b)
    (if (> a b)
        1
        (* (term a)
           (product term (next a) next b))))

(define (factorial n)
    (product identity 1 inc n))
(factorial 5)

120

In [44]:
(define (wallis-term k)
    (if (even? k) (/ (+ k 2) (+ k 1))
                  (/ (+ k 1) (+ k 2))))
(define (wallis-pi n)
    (* 4 (product wallis-term 1 inc n)))
(exact->inexact (wallis-pi 100))

3.1570301764551676

In [45]:
;; b
(define (product term a next b)
    (define (iter a result)
        (if (> a b)
            result
            (iter (next a) (* result (term a)))))
    (iter a 1))

(define (factorial n)
    (product identity 1 inc n))
(factorial 5)

120

## Q1.32

In [46]:
;; a
(define (accumulate combiner null-value term a next b)
    (if (> a b)
        null-value
        (combiner (term a)
                  (accumulate combiner null-value term (next a) next b))))
(define (sum term a next b)
    (accumulate + 0 term a next b))
(define (product term a next b)
    (accumulate * 1 term a next b))
(define (identity k) k)
(define (inc k) (+ k 1))

(display (sum identity 1 inc 5)) (newline)
(display (product identity 1 inc 5))

15
120

In [47]:
;; b
(define (accumulate combiner null-value term a next b)
    (if (> a b)
        null-value
        (combiner (term a)
                  (accumulate combiner null-value term (next a) next b))))
(define (accumulate combiner null-value term a next b)
    (define (iter a result)
        (if (> a b)
            result
            (iter (next a) (combiner result (term a)))))
    (iter a null-value))
(define (sum term a next b)
    (accumulate + 0 term a next b))
(define (product term a next b)
    (accumulate * 1 term a next b))
(define (identity k) k)
(define (inc k) (+ k 1))

(display (sum identity 1 inc 5)) (newline)
(display (product identity 1 inc 5))

15
120

## Q1.33

In [48]:
;; a
(define (filtered-accumulate combiner null-value predicate term a next b)
    (if (> a b)
        null-value
        (if (predicate a)
            (combiner (term a)
                      (filtered-accumulate combiner null-value predicate term (next a) next b))
            (combiner null-value
                      (filtered-accumulate combiner null-value predicate term (next a) next b)))))

(define (identity k) k)
(define (inc k) (+ k 1))
(define (square x) (* x x))
(define (prime? n)
    (= n (smallest-divisor n)))
(define (smallest-divisor n) (find-divisor n 2))
(define (find-divisor n test-divisor)
    (cond ((> (square test-divisor) n) n)
          ((divides? test-divisor n) test-divisor)
          (else (find-divisor n (+ test-divisor 1)))))
(define (divides? a b) (= (remainder b a) 0))

(filtered-accumulate + 0 prime? square 1 inc 5)  ; 2^2+3^2+5^2 = 39

39

In [49]:
;; b
(define (gcd a b)
    (if (= b 0)
        a
        (gcd b (remainder a b))))
(define (rel-prime? i n)
    (= (gcd i n) 1))
(define (relprime-product n)  
    (define (rel-prime-partial? i)
        (rel-prime? i n))
    (filtered-accumulate * 1 rel-prime-partial? identity 1 inc n))

(display (gcd 2 10)) (newline)
(display (gcd 3 10)) (newline)
(display (gcd 4 10)) (newline)
(display (gcd 5 10)) (newline)
(display (gcd 6 10)) (newline)
(display (gcd 7 10)) (newline)
(display (gcd 8 10)) (newline)
(display (gcd 9 10)) (newline)
(relprime-product 10)  ; 3*7*9 = 189

2
1
2
5
2
1
2
1


189

## Q1.34

`(f f)` becomes `(f 2)` but `2` is not a function so the subsequent call `(2 2)` will yield an error.

## Q1.35

$$
\begin{align}
1+\frac{1}{\phi} &= 1+\frac{2}{1+\sqrt{5}} \\ 
&= \frac{3+\sqrt{5}}{1+\sqrt{5}} \\
&= \frac{3+\sqrt{5}}{1+\sqrt{5}} \left( \frac{1-\sqrt{5}}{1-\sqrt{5}} \right) \\
&= \frac{2-2\sqrt{5}+1-5}{1-5} \\
&= \frac{-2\sqrt{5}-2}{-4} \\
&= \frac{1+\sqrt{5}}{2} = \phi
\end{align}
$$.

In [50]:
(define tolerance 0.00001)
(define (fixed-point f first-guess)
    (define (close-enough? v1 v2)
        (< (abs (- v1 v2)) tolerance))
    (define (try guess)
        (let ((next (f guess)))
            (if (close-enough? guess next)
                next
                (try next))))
    (try first-guess))
(define (average x y)
    (/ (+ x y) 2))
(define (phi-est x)
    (fixed-point (lambda (y) (average y (+ 1 (/ 1 y))))
                 x))
(phi-est 1.0)

1.6180311591702674

## Q1.36

In [51]:
(define tolerance 0.00001)
(define (fixed-point f first-guess)
    (define (close-enough? v1 v2)
        (< (abs (- v1 v2)) tolerance))
    (define (try guess)
        (display guess) (newline)
        (let ((next (f guess)))
            (if (close-enough? guess next)
                next
                (try next))))
    (try first-guess))
(define (average x y)
    (/ (+ x y) 2))
(define (x-to-x-est x)
    (fixed-point (lambda (y) (average y (/ (log 1000) (log y))))
                 x))
(define (x-to-x-nodamp-est x)
    (fixed-point (lambda (y) (/ (log 1000) (log y)))
                 x))

(x-to-x-est 2.0)
(newline)
(x-to-x-nodamp-est 2.0)

2.0
5.9828921423310435
4.922168721308343
4.628224318195455
4.568346513136242
4.5577305909237005
4.555909809045131
4.555599411610624
4.5555465521473675

2.0
9.965784284662087
3.004472209841214
6.279195757507157
3.759850702401539
5.215843784925895
4.182207192401397
4.8277650983445906
4.387593384662677
4.671250085763899
4.481403616895052
4.6053657460929
4.5230849678718865
4.577114682047341
4.541382480151454
4.564903245230833
4.549372679303342
4.559606491913287
4.552853875788271
4.557305529748263
4.554369064436181
4.556305311532999
4.555028263573554
4.555870396702851
4.555315001192079
4.5556812635433275
4.555439715736846
4.555599009998291
4.555493957531389
4.555563237292884
4.555517548417651
4.555547679306398
4.555527808516254
4.555540912917957


4.555532270803653

Average damping speeds convergence.

## Q1.37

In [52]:
;; a
(define sqrt5 (expt 5 0.5))
(define phi (/ (+ 1 sqrt5) 2))

(define (cont-frac n d k)
    (define (cont-frac-term i)
        (if (= i (+ k 1))
            0
            (/ (n i) (+ (d i) (cont-frac-term (+ i 1))))))
    (cont-frac-term 1))

(display (/ 1 phi)) (newline)
(display (cont-frac (lambda (i) 1.0) (lambda (i) 1.0) 9)) (newline)
(display (cont-frac (lambda (i) 1.0) (lambda (i) 1.0) 10)) (newline)

0.6180339887498948
0.6181818181818182
0.6179775280898876


We get 4 decimals of accuracy at `k=10`.

In [53]:
;; b
(define sqrt5 (expt 5 0.5))
(define phi (/ (+ 1 sqrt5) 2))

(define (cont-frac n d k)
    (define (cont-frac-iter i acc)
        (if (= i 0)
            acc
            (cont-frac-iter (- i 1) (/ (n i) (+ acc (d i))))))
    (cont-frac-iter k 0))

(display (/ 1 phi)) (newline)
(display (cont-frac (lambda (i) 1.0) (lambda (i) 1.0) 9)) (newline)
(display (cont-frac (lambda (i) 1.0) (lambda (i) 1.0) 10)) (newline)

0.6180339887498948
0.6181818181818182
0.6179775280898876


## Q1.38

In [54]:
(define (cont-frac n d k)
    (define (cont-frac-iter i acc)
        (if (= i 0)
            acc
            (cont-frac-iter (- i 1) (/ (n i) (+ acc (d i))))))
    (cont-frac-iter k 0))
(define (d i)
    (let ((i+1 (+ i 1)))
        (cond ((= (remainder i+1 3) 0)
                  (* (/ i+1 3) 2))
              (else 1))))

(display (exp 1))
(+ 2 (cont-frac (lambda (i) 1.0) d 10))

2.718281828459045

2.7182817182817183

## Q1.39

In [55]:
(define (cont-frac n d k)
    (define (cont-frac-iter i acc)
        (if (= i 0)
            acc
            (cont-frac-iter (- i 1) (/ (n i) (+ acc (d i))))))
    (cont-frac-iter k 0))
(define (tan-cf x k)
    (define (n i)
        (cond ((= i 1) x)
              (else (* -1 x x))))
    (define (d i)
        (- (* 2 i) 1))
    (cont-frac n d k))
(define pi (acos -1))

(let ((x1 0)
      (x2 0.7)
      (x3 pi)
      (x4 (/ pi 4))
      (x5 -1.2))
    (display (tan x1)) (display " ") (display (tan-cf x1 10)) (newline)
    (display (tan x2)) (display " ") (display (tan-cf x2 10)) (newline)
    (display (tan x3)) (display " ") (display (tan-cf x3 10)) (newline)
    (display (tan x4)) (display " ") (display (tan-cf x4 10)) (newline)
    (display (tan x5)) (display " ") (display (tan-cf x5 10)) (newline))

0 0
0.8422883804630793 0.8422883804630793
-1.2246467991473532e-16 -1.893214149359168e-9
0.9999999999999999 1.0
-2.5721516221263183 -2.5721516221263188


## Q1.40

In [56]:
(define (fixed-point f first-guess)
    (define (close-enough? v1 v2)
        (< (abs (- v1 v2)) tolerance))
    (define (try guess)
        (let ((next (f guess)))
            (if (close-enough? guess next)
                next
                (try next))))
    (try first-guess))
(define dx 0.00001)
(define (square x) (* x x))
(define (cube x) (* x x x))
(define (deriv g)
    (lambda (x) (/ (- (g (+ x dx)) (g x)) dx)))
(define (newton-transform g)
    (lambda (x) (- x (/ (g x) ((deriv g) x)))))
(define (newtons-method g guess)
    (fixed-point (newton-transform g) guess))

(define (cubic a b c)
    (lambda (y)
        (+ (cube y)
           (* a (square y))
           (* b y)
           c)))

(display (newtons-method (cubic 0 0 8) 1)) (newline)  ; x^3+8=0 -> x=-2
(display (newtons-method (cubic 1 1 0) 1)) (newline)  ; x^3+x^2+x=0 -> x=0
(display (newtons-method (cubic 0 -1 -6) 1)) (newline)  ; x^3+x-2=0 -> x=2

-1.999999999999997
9.898106050336853e-12
2.000000000000419


## Q1.41

In [57]:
(define (double f)
    (lambda (x) (f (f x))))
(define (inc x) (+ x 1))
(((double (double double)) inc) 5)  ; (2^2)^2 + 5 = 21

21

## Q1.42

In [58]:
(define (compose f g)
    (lambda (x) (f (g x))))
((compose square inc) 6)

49

## Q1.43

In [59]:
(define (repeated f times)
    (if (= times 1)
        f
        (compose (repeated f (- times 1)) f)))
((repeated square 2) 5)

625

## Q1.44

In [60]:
(define dx 0.00001) 
(define (smooth f)
    (lambda (x)
        (/ (+ (f (- x dx))
              (f x)
              (f (+ x dx)))
           3)))
(display ((smooth square) 3)) (newline)
(display ((smooth (smooth square)) 3)) (newline)
(display (((repeated smooth 2) square) 3))

9.000000000066665
9.000000000133333
9.000000000133333

## Q1.45

In [61]:
(define tolerance 0.00001)
(define (fixed-point f first-guess)
    (define (close-enough? v1 v2)
        (< (abs (- v1 v2)) tolerance))
    (define (try guess)
        (let ((next (f guess)))
            (if (close-enough? guess next)
                next
                (try next))))
    (try first-guess))
(define (average x y)
    (/ (+ x y) 2))
(define (average-damp f)
    (lambda (x) (average x (f x))))
(define (nth-rt-est n damps x)
    (fixed-point ((repeated average-damp damps) (lambda (y) (/ x (expt y (- n 1)))))
                 x))

(display (nth-rt-est 4 2 4.0)) (newline)
(display (nth-rt-est 7 2 4.0)) (newline)
(display (nth-rt-est 8 3 4.0)) (newline)

1.4142135623730954
1.2190094924173325
1.18920711502059


We need $log_{2}n$ damps to compute the $n$th root.

This is because our fixed-point search is essentially implementing Newton's method using a modified formula that successive average damping corrects.

To see this, the Newton's method update rule is
$$
y_{n+1} = y_n - \frac{f(y)}{f'(y)}
$$.

Consider finding the square root of $x$ by finding the zeroes of $f(y) = y^2 - x$.

Our once-damped algorithm computes
$$
y_{n+1} = \frac{1}{2}\left( y_n + \frac{x}{y_n} \right) = \frac{y_n^2 + x}{2y_n}
$$.

Newton's method would have us compute
$$
y_{n+1} = y_n - \frac{y_n^2-x}{2y_n} = \frac{y_n^2+x}{2y_n}
$$
which is identical.

Now consider $g(y) = y^4 - x$ for computing the 4th root.

Newton's method would have us compute
$$
y_{n+1} = y_n - \frac{y_n^4-x}{4y_n^3} = \frac{3y_n^4+x}{4y_n^3}
$$.

Our once-damped algorithm would compute
$$
y_{n+1} = \frac{1}{2}\left( y_n + \frac{x}{y_n^3} \right) = \frac{y_n^4 + x}{2y_n^3}
$$
, but by damping again, we'd instead compute
$$
y_{n+1} = \frac{1}{2}\left( y_n + \frac{y_n^4 + x}{2y_n^3} \right) = \frac{3y_n^4 + x}{4y_n^3}
$$
to match Newton's method.

Each successive dampening introduces a factor of 2 in the demoniator which motivates using $log_{2}n$ damps for the $n$th root.

Note though that this doesn't answer the question of why our "insufficiently damped" algorithm still works for computing cube roots, for instance.

In [62]:
(define (nth-rt-est-auto n x)
    (let ((damps (floor (log n 2))))
        (nth-rt-est n damps x)))

(display (nth-rt-est-auto 4 4.0)) (newline)
(display (nth-rt-est-auto 7 4.0)) (newline)
(display (nth-rt-est-auto 8 4.0)) (newline)

1.4142135623730954
1.2190094924173325
1.18920711502059


## Q1.46

In [63]:
(define (iterative-improve good-enough? improve)
    (lambda (x)
       (if (good-enough? x)
           x
           ((iterative-improve good-enough? improve) (improve x)))))

(define (square x) (* x x))
(define (average x y)
    (/ (+ x y) 2))
(define (sqrt-est x)
    (define (good-root? guess)
        (< (abs (- (square guess) x)) 0.001))
    (define (improve-root guess)
        (average guess (/ x guess)))
    ((iterative-improve good-root? improve-root) x))

(sqrt-est 2.0)

1.4142156862745097

In [64]:
(define (fixed-point f first-guess)
    (define tolerance 0.00001)
    (define (good-fp? x)
        (define (close-enough? v1 v2)
                (< (abs (- v1 v2)) tolerance))
        (close-enough? x (f x)))
    (define (improve-fp guess)
        (define (average x y)
            (/ (+ x y) 2))
        (define (average-damp f)
            (lambda (x) (average x (f x))))
        ((average-damp f) guess))
    ((iterative-improve good-fp? improve-fp) 2.0))

(fixed-point (lambda (y) (/ 2.0 y)) 1.0)

1.4142156862745097